# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [4]:
#Your code here

import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [8]:
df.shape

(8188, 4)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [11]:
df.action.unique()

array(['view', 'click'], dtype=object)

In [12]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [26]:
# explore number of viewers, number of clickers, number of viewers who did not click,
# number of clickers who did not view

vids = set(df[df.action=='view']['id'].unique())
cids = set(df[df.action=='click']['id'].unique())

print("Number of viewers: {} \tNumber of clickers: {}".format(len(vids), len(cids)))
print("Number of viewers who did not click: {}".format(len(vids-cids)))
print("Number of clickers who did not view: {}".format(len(cids-vids)))


Number of viewers: 6328 	Number of clickers: 1860
Number of viewers who did not click: 4468
Number of clickers who did not view: 0


In [41]:
# Overlap of experiment and control groups
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())

print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
#Your code here
#H_O: The experimental page did not increase the number of clicks relative to the control group. 
# Experimental page, Number of clicks <= 1860
#H_A: The experimental page increased number of clicks > 1860   




In [44]:
# Make data more usable for this exercise. Make a df for each group.
experiment_group = df[df.group=='experiment'].copy()

# Convert qualitative value to numerical value
experiment_group['dummy'] = 1

# Pivot to group view in rows/wider format (eg, some ID took 2 actions--view and click)
experiment = experiment_group.pivot(index = 'id', columns = 'action', values = 'dummy')
experiment.fillna(0, inplace = True)
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [46]:
control_group = df[df.group=='control'].copy()

control_group['dummy'] = 1

control = control_group.pivot(index = 'id', columns = 'action', values = 'dummy')
control.fillna(0, inplace = True)
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [47]:
# evaluate sample size
len(experiment), len(control)

(2996, 3332)

In [48]:
# get click-through rate
experiment['click'].mean(), control['click'].mean()

(0.3097463284379172, 0.2797118847539016)

In [50]:
# do statistical test to see if stat significant
# t test

from scipy.stats import ttest_ind

In [51]:
ttest_ind(control['click'], experiment['click'])

Ttest_indResult(statistic=-2.6195696844542202, pvalue=0.008825098914958293)

In [ ]:
# p-value < 0.01, so result is stat significant

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [59]:
#Your code here
expected_number_of_clicks = len(experiment) * control['click'].mean()
expected_number_of_clicks

838.0168067226891

In [54]:
experiment['click'].sum()

928.0

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [55]:
import numpy as np

In [60]:
#Your code here: referring to formula above

var = len(experiment) * control['click'].mean() * (1 - control['click'].mean())
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [61]:
#Your code here
z = (experiment['click'].sum() - expected_number_of_clicks) / std
z

3.6625360854823588

In [62]:
from scipy.stats import norm

In [63]:
norm.sf(z) # very small p-value, so indicates that result is statistically significant

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
# Comment: Yes, while the p-value is slightly lower, 
# both would lead to confident rejection of the null hypothesis. 
# The experimental page appears to be a more effective design.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.